!pip install -U numpy

import sys
print(sys.path) 
print(sys.executable)

import sys 
!{sys.executable} -m pip install nltk

# Import Packages

In [1]:
import pandas as pd
import numpy as np 
import csv
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings("ignore")


# Dataset Cleaning

In [2]:
path = '/home/manikya_varshney/Documents/Python/Yale/Data/h01-20200912-101538.csv'
data = pd.read_csv(path, low_memory=False)

In [3]:
data.shape

(19966, 79)

In [4]:
cols = [1, 3, 4, 5,6, 7, 8, 9, 10, 11, 17, 18, 19, 22, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 54, 55, 56, 58, 63, 64, 65, 67, 76, 77]

In [5]:
data = data[data.columns.values[cols]]

In [6]:
data.shape

(19966, 38)

In [7]:
data.to_csv('data_filtered_h01-20200912-101538.csv',index=False)

In [8]:
data['lang'].value_counts()

en     10043
es      3085
ja      2151
pt      1251
hi       645
fr       458
in       450
und      398
ko       213
tr       182
ar       177
de       149
th       145
ca       132
it       113
tl       101
nl        59
zh        46
mr        28
ne        18
et        16
pl        15
cs        11
el        11
ro         9
ta         7
da         5
lt         4
bn         4
gu         4
ru         3
no         3
fa         3
eu         3
ht         3
cy         3
sv         3
ur         2
vi         2
iw         2
fi         2
or         2
kn         2
te         1
sl         1
lv         1
Name: lang, dtype: int64

In [9]:
data[data['lang'] == 'en'].shape

(10043, 38)

In [10]:
data_en = data[data['lang'] == 'en'].reset_index(drop=True)

In [11]:
data_en

id  is_quote_tweet  quoted_tweet_id  \
0      1.304786e+18            True     1.304732e+18   
1      1.304786e+18            True     1.304631e+18   
2      1.304786e+18           False              NaN   
3      1.304786e+18           False              NaN   
4      1.304786e+18            True     1.304775e+18   
...             ...             ...              ...   
10038  1.304787e+18           False              NaN   
10039  1.304787e+18           False              NaN   
10040  1.304787e+18            True     1.304093e+18   
10041  1.304787e+18           False              NaN   
10042  1.304787e+18           False              NaN   

       in_reply_to_status_id_str  in_reply_to_user_id_str  \
0                            NaN                      NaN   
1                            NaN                      NaN   
2                            NaN                      NaN   
3                            NaN                      NaN   
4                            NaN                      NaN   
...                          ...                      ...   
10038                        NaN                      NaN   
10039                        NaN                      NaN   
10040                        NaN                      NaN   
10041                        NaN                      NaN   
10042                        NaN                      NaN   

      in_reply_to_screen_name lang  quote_count  reply_count  retweet_count  \
0                         NaN   en            0            0              0   
1                         NaN   en            0            0              0   
2                         NaN   en            0            0              0   
3                         NaN   en            0            0              0   
4                         NaN   en            0            0              0   
...                       ...  ...          ...          ...            ...   
10038                     NaN   en            0            0              0   
10039                     NaN   en            0            0              0   
10040                     NaN   en            0            0              0   
10041                     NaN   en            0            0              0   
10042                     NaN   en            0            0              0   

       ...                                            RT_text  \
0      ...                                                NaN   
1      ...  This is why many of us have cautioned from the...   
2      ...  Let me get this right, it was okay for Steven ...   
3      ...                                                NaN   
4      ...                                                NaN   
...    ...                                                ...   
10038  ...  Have you struggled with your eating habits sin...   
10039  ...  I looked up the definition of corona in the Ch...   
10040  ...  Fun fact: in 2008 this officer pistol whipped ...   
10041  ...  a stay at home challenge to keep people safe, ...   
10042  ...  Deep State Fauci Calls for Economic Lockdown U...   

                                            RT_full_text         QT_id  \
0                                                    NaN  1.304732e+18   
1      This is why many of us have cautioned from the...  1.304631e+18   
2      Let me get this right, it was okay for Steven ...           NaN   
3                                                    NaN           NaN   
4                                                    NaN  1.304775e+18   
...                                                  ...           ...   
10038                                                NaN           NaN   
10039                                                NaN           NaN   
10040  Fun fact: in 2008 this officer pistol whipped ...  1.304093e+18   
10041  a stay at home challenge to keep people safe, ...           NaN   
10042  Deep State Fauci Calls for Economic Lockdown U...           

In [12]:
data_en['RT_Flag'] = data_en['text'].str[:2]=='RT'

In [13]:
data_en.shape

(10043, 39)

##### is_quote_tweet --> False --> Non RT

In [14]:
interim = data_en[np.where((data_en['is_quote_tweet']==False) & (data_en['RT_Flag']==False),True,False)].reset_index(drop=True)

In [15]:
interim['extended_tweet_full_text'] = np.where(~interim['truncated'],interim['text'],interim['extended_tweet_full_text'])

In [16]:
interim.shape

(2288, 39)

In [17]:
non_rep = interim[['id','user_id','extended_tweet_full_text']]

In [18]:
non_rep.shape

(2288, 3)

In [19]:
non_rep.drop_duplicates(['id']).shape

(2287, 3)

In [20]:
non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

##### is_quote_tweet --> False --> RT

In [21]:
interim = data_en[np.where((data_en['is_quote_tweet']== False) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

In [22]:
interim.shape

(5182, 39)

In [23]:
interim[['RT_text','RT_full_text']]

RT_text  \
0     Let me get this right, it was okay for Steven ...   
1     Hospitals in the United States have turned dow...   
2     My friends parents have died from Covid &amp; ...   
3     📢 Our 7-part series on life after lockdown ‘EM...   
4     @JoeTalkShow I wrote a tweet after u came back...   
...                                                 ...   
5177  A Perth woman in quarantine has become the fir...   
5178  Have you struggled with your eating habits sin...   
5179  I looked up the definition of corona in the Ch...   
5180  a stay at home challenge to keep people safe, ...   
5181  Deep State Fauci Calls for Economic Lockdown U...   

                                           RT_full_text  
0     Let me get this right, it was okay for Steven ...  
1     Hospitals in the United States have turned dow...  
2     My friends parents have died from Covid &amp; ...  
3     📢 Our 7-part series on life after lockdown ‘EM...  
4     @JoeTalkShow I wrote a tweet after u came back...  
...                                                 ...  
5177  A Perth woman in quarantine has become the fir...  
5178                                                NaN  
5179                                                NaN  
5180  a stay at home challenge to keep people safe, ...  
5181  Deep State Fauci Calls for Economic Lockdown U...  

[5182 rows x 2 columns]

In [24]:
interim['RT_full_text'] = np.where(interim['RT_full_text'].isna(),interim['RT_text'],interim['RT_full_text'])

In [25]:
interim = interim[['RT_id','RT_user_id','RT_full_text']]

In [26]:
interim.columns = non_rep.columns

In [27]:
interim.shape

(5182, 3)

In [28]:
interim = interim.drop_duplicates(['id','user_id']).reset_index(drop=True)

In [29]:
interim.shape

(2221, 3)

In [30]:
non_rep = non_rep.append(interim,ignore_index=True)

In [31]:
non_rep.shape

(4508, 3)

In [32]:
non_rep['id'].value_counts()

1.304786e+18    2
1.304786e+18    2
1.304786e+18    2
1.304786e+18    2
1.304786e+18    2
               ..
1.304787e+18    1
1.304786e+18    1
1.304786e+18    1
1.304787e+18    1
1.304705e+18    1
Name: id, Length: 4444, dtype: int64

In [33]:
non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

In [34]:
non_rep.shape

(4444, 3)

##### is_quote_tweet --> True --> Non RT

In [35]:
interim = data_en[np.where((data_en['is_quote_tweet']== True) & (data_en['RT_Flag']== False),True,False)].reset_index(drop=True)

In [36]:
interim.shape

(758, 39)

In [37]:
interim['extended_tweet_full_text'] = np.where(~interim['truncated'],interim['text'],interim['extended_tweet_full_text'])

In [38]:
interim[['text','extended_tweet_full_text']]

text  \
0                         What i gain in this pandemic   
1                 My baby school got them wearing both   
2    And it should not be radical. \n\nMilitary and...   
3    No brainer, it's all political &amp; it's not ...   
4    In case you haven't noticed, we're in the midd...   
..                                                 ...   
753  And still nothing is done to stop Merchant Cit...   
754                                  Betul............   
755  @ScottMorrisonMP  are proud of this happening ...   
756  #BlackLivesMatter Andy. Words matter, especial...   
757  Excellent visual of the numbers of the R0 for ...   

                              extended_tweet_full_text  
0                         What i gain in this pandemic  
1                 My baby school got them wearing both  
2    And it should not be radical. \n\nMilitary and...  
3    No brainer, it's all political &amp; it's not ...  
4    In case you haven't noticed, we're in the midd...  
..                                                 ...  
753  And still nothing is done to stop Merchant Cit...  
754                                  Betul............  
755  @ScottMorrisonMP  are proud of this happening ...  
756  #BlackLivesMatter Andy. Words matter, especial...  
757  Excellent visual of the numbers of the R0 for ...  

[758 rows x 2 columns]

In [39]:
interim = interim[['id','user_id','extended_tweet_full_text']]

In [40]:
interim.columns = non_rep.columns

In [41]:
interim.shape

(758, 3)

In [42]:
interim

id       user_id  \
0    1.304786e+18  1.310861e+08   
1    1.304786e+18  2.996246e+09   
2    1.304786e+18  1.034838e+18   
3    1.304786e+18  5.273566e+08   
4    1.304786e+18  1.292052e+09   
..            ...           ...   
753  1.304787e+18  3.310412e+09   
754  1.304787e+18  2.838848e+09   
755  1.304787e+18  1.097800e+18   
756  1.304787e+18  1.022819e+18   
757  1.304787e+18  2.555459e+08   

                              extended_tweet_full_text  
0                         What i gain in this pandemic  
1                 My baby school got them wearing both  
2    And it should not be radical. \n\nMilitary and...  
3    No brainer, it's all political &amp; it's not ...  
4    In case you haven't noticed, we're in the midd...  
..                                                 ...  
753  And still nothing is done to stop Merchant Cit...  
754                                  Betul............  
755  @ScottMorrisonMP  are proud of this happening ...  
756  #BlackLivesMatter Andy. Words matter, especial...  
757  Excellent visual of the numbers of the R0 for ...  

[758 rows x 3 columns]

In [43]:
interim = interim.drop_duplicates(['id','user_id']).reset_index(drop=True)

In [44]:
interim.shape

(758, 3)

In [45]:
non_rep = non_rep.append(interim,ignore_index=True)

In [46]:
non_rep.shape

(5202, 3)

In [47]:
non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

In [48]:
non_rep.shape

(5200, 3)

In [49]:
interim = data_en[np.where((data_en['is_quote_tweet']== True) & (data_en['RT_Flag']== False),True,False)].reset_index(drop=True)

In [50]:
interim.shape

(758, 39)

In [51]:
interim[['QT_text','QT_full_text']]

QT_text  \
0           R.I.P to my unhealthy phone charging habit   
1          So does mask or a face shield work better 🤣   
2    BERNIE BUCKS: Sanders Says ‘Every Working Clas...   
3    Do you think some STATE SHUTDOWNS are directly...   
4    “The RNC says it has knocked on the doors of 1...   
..                                                 ...   
753  BREAKING: 221 people have tested positive for ...   
754  Kalau tidak py wacana perbaikan soal penangana...   
755  This is what’s occurring in Australia @ScottMo...   
756  Every life matters and this virus is our enemy...   
757  @Hood_Biologist Just to give a quick compariso...   

                                          QT_full_text  
0                                                  NaN  
1                                                  NaN  
2                                                  NaN  
3                                                  NaN  
4    “The RNC says it has knocked on the doors of 1...  
..                                                 ...  
753  BREAKING: 221 people have tested positive for ...  
754  Kalau tidak py wacana perbaikan soal penangana...  
755  This is what’s occurring in Australia @ScottMo...  
756  Every life matters and this virus is our enemy...  
757  @Hood_Biologist Just to give a quick compariso...  

[758 rows x 2 columns]

In [52]:
interim['QT_full_text'] = np.where(interim['QT_full_text'].isna(),interim['QT_text'],interim['QT_full_text'])

In [53]:
interim.shape

(758, 39)

In [54]:
interim[['QT_text','QT_full_text']]

QT_text  \
0           R.I.P to my unhealthy phone charging habit   
1          So does mask or a face shield work better 🤣   
2    BERNIE BUCKS: Sanders Says ‘Every Working Clas...   
3    Do you think some STATE SHUTDOWNS are directly...   
4    “The RNC says it has knocked on the doors of 1...   
..                                                 ...   
753  BREAKING: 221 people have tested positive for ...   
754  Kalau tidak py wacana perbaikan soal penangana...   
755  This is what’s occurring in Australia @ScottMo...   
756  Every life matters and this virus is our enemy...   
757  @Hood_Biologist Just to give a quick compariso...   

                                          QT_full_text  
0           R.I.P to my unhealthy phone charging habit  
1          So does mask or a face shield work better 🤣  
2    BERNIE BUCKS: Sanders Says ‘Every Working Clas...  
3    Do you think some STATE SHUTDOWNS are directly...  
4    “The RNC says it has knocked on the doors of 1...  
..                                                 ...  
753  BREAKING: 221 people have tested positive for ...  
754  Kalau tidak py wacana perbaikan soal penangana...  
755  This is what’s occurring in Australia @ScottMo...  
756  Every life matters and this virus is our enemy...  
757  @Hood_Biologist Just to give a quick compariso...  

[758 rows x 2 columns]

In [55]:
interim = interim[['QT_id','QT_user_id','QT_full_text']]

In [56]:
interim.columns = non_rep.columns

In [57]:
interim

id       user_id  \
0    1.304732e+18  1.074866e+18   
1    1.304775e+18  2.508580e+07   
2    1.304543e+18  4.163452e+07   
3    1.304783e+18  4.765364e+09   
4    1.304774e+18  1.113425e+07   
..            ...           ...   
753  1.304772e+18  1.568751e+07   
754  1.304775e+18  3.254877e+09   
755  1.304754e+18  1.292287e+18   
756  1.304783e+18  1.192550e+18   
757  1.304785e+18  9.679572e+17   

                              extended_tweet_full_text  
0           R.I.P to my unhealthy phone charging habit  
1          So does mask or a face shield work better 🤣  
2    BERNIE BUCKS: Sanders Says ‘Every Working Clas...  
3    Do you think some STATE SHUTDOWNS are directly...  
4    “The RNC says it has knocked on the doors of 1...  
..                                                 ...  
753  BREAKING: 221 people have tested positive for ...  
754  Kalau tidak py wacana perbaikan soal penangana...  
755  This is what’s occurring in Australia @ScottMo...  
756  Every life matters and this virus is our enemy...  
757  @Hood_Biologist Just to give a quick compariso...  

[758 rows x 3 columns]

In [58]:
interim = interim.drop_duplicates(['id','user_id']).reset_index(drop=True)

In [59]:
interim.shape

(553, 3)

In [60]:
non_rep = non_rep.append(interim,ignore_index=True)

In [61]:
non_rep.shape

(5753, 3)

In [62]:
non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

In [63]:
non_rep.shape

(5588, 3)

##### is_quote_tweet --> True --> RT

In [64]:
data_en[np.where((data_en['is_quote_tweet']== True) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

id  is_quote_tweet  quoted_tweet_id  \
0     1.304786e+18            True     1.304631e+18   
1     1.304786e+18            True     1.304568e+18   
2     1.304786e+18            True     1.304603e+18   
3     1.304786e+18            True     1.304626e+18   
4     1.304786e+18            True     1.304341e+18   
...            ...             ...              ...   
1810  1.304787e+18            True     1.304503e+18   
1811  1.304787e+18            True     1.304756e+18   
1812  1.304787e+18            True     1.304610e+18   
1813  1.304787e+18            True     1.304093e+18   
1814  1.304787e+18            True     1.304093e+18   

      in_reply_to_status_id_str  in_reply_to_user_id_str  \
0                           NaN                      NaN   
1                           NaN                      NaN   
2                           NaN                      NaN   
3                           NaN                      NaN   
4                           NaN                      NaN   
...                         ...                      ...   
1810                        NaN                      NaN   
1811                        NaN                      NaN   
1812                        NaN                      NaN   
1813                        NaN                      NaN   
1814                        NaN                      NaN   

     in_reply_to_screen_name lang  quote_count  reply_count  retweet_count  \
0                        NaN   en            0            0              0   
1                        NaN   en            0            0              0   
2                        NaN   en            0            0              0   
3                        NaN   en            0            0              0   
4                        NaN   en            0            0              0   
...                      ...  ...          ...          ...            ...   
1810                     NaN   en            0            0              0   
1811                     NaN   en            0            0              0   
1812                     NaN   en            0            0              0   
1813                     NaN   en            0            0              0   
1814                     NaN   en            0            0              0   

      ...                                       RT_full_text         QT_id  \
0     ...  This is why many of us have cautioned from the...  1.304631e+18   
1     ...                                                NaN  1.304568e+18   
2     ...  Folks, literally for week ending Sep. 5, the P...  1.304603e+18   
3     ...                                                NaN  1.304626e+18   
4     ...                                                NaN  1.304341e+18   
...   ...                                                ...           ...   
1810  ...                                                NaN  1.304503e+18   
1811  ...  Completely forgot about the Louisville escorts...  1.304756e+18   
1812  ...  Good morning. As 1,000+ Americans die a day, a...  1.304610e+18   
1813  ...  Fun fact: in 2008 this officer pistol whipped ...  1.304093e+18   
1814  ...  Fun fact: in 2008 this officer pistol whipped ...  1.304093e+18   

      QT_lang    QT_user_id      QT_username               QT_name  \
0          en  8.312794e+17  Craig_A_Spencer  Craig Spencer MD MPH   
1          en  2.949916e+08  MindfulMajority      Mindful Majority   
2          en  4.445154e+07          kylamb8             Kyle Lamb   
3          en  3.287109e+07     kylegriffin1          Kyle Griffin   
4          en  7.585442e+07         konkonas     Konkona Sensharma   
...       ...           ...              ...                   ...   
1810       en  1.501249e+07          CBSNews              CBS News   
1811       en  2.215891e+09  LouClinicEscort   LvilleClinicEscorts   
1812       en  1.975016e+07    justinhendrix        Justin Hendrix   
1813       en  8.863983e+17   davenewworld_2  Fifty Shades o